# Paper 10: Deep Residual Learning for Image Recognition
## Kaiming He, Xiangyu Zhang, Shaoqing Ren, Jian Sun (2015)

### ResNet: Skip Connections Enable Very Deep Networks

ResNet introduced residual connections that allow training networks with 100+ layers.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

np.random.seed(42)

## The Problem: Degradation in Deep Networks

Before ResNet, adding more layers actually made networks worse (not due to overfitting, but optimization difficulty).

In [ ]:
def relu(x):
    return np.maximum(0, x)

def relu_derivative(x):
    return (x > 0).astype(float)

class PlainLayer:
    """Standard neural network layer"""
    def __init__(self, input_size, output_size):
        self.W = np.random.randn(output_size, input_size) * np.sqrt(2.0 / input_size)
        self.b = np.zeros((output_size, 1))
    
    def forward(self, x):
        self.x = x
        self.z = np.dot(self.W, x) + self.b
        self.a = relu(self.z)
        return self.a
    
    def backward(self, dout):
        da = dout * relu_derivative(self.z)
        self.dW = np.dot(da, self.x.T)
        self.db = np.sum(da, axis=1, keepdims=True)
        dx = np.dot(self.W.T, da)
        return dx

class ResidualBlock:
    """Residual block with skip connection: y = F(x) + x"""
    def __init__(self, size):
        self.layer1 = PlainLayer(size, size)
        self.layer2 = PlainLayer(size, size)
    
    def forward(self, x):
        self.x = x
        
        # Residual path F(x)
        out = self.layer1.forward(x)
        out = self.layer2.forward(out)
        
        # Skip connection: F(x) + x
        self.out = out + x
        return self.out
    
    def backward(self, dout):
        # Gradient flows through both paths
        # Skip connection provides direct path
        dx_residual = self.layer2.backward(dout)
        dx_residual = self.layer1.backward(dx_residual)
        
        # Total gradient: residual path + skip connection
        dx = dx_residual + dout  # This is the key!
        return dx

print("ResNet components initialized")

## Build Plain Network vs ResNet

In [ ]:
class PlainNetwork:
    """Plain deep network without skip connections"""
    def __init__(self, input_size, hidden_size, num_layers):
        self.layers = []
        
        # First layer
        self.layers.append(PlainLayer(input_size, hidden_size))
        
        # Hidden layers
        for _ in range(num_layers - 2):
            self.layers.append(PlainLayer(hidden_size, hidden_size))
        
        # Output layer
        self.layers.append(PlainLayer(hidden_size, input_size))
    
    def forward(self, x):
        for layer in self.layers:
            x = layer.forward(x)
        return x
    
    def backward(self, dout):
        for layer in reversed(self.layers):
            dout = layer.backward(dout)
        return dout

class ResidualNetwork:
    """Deep network with residual connections"""
    def __init__(self, input_size, hidden_size, num_blocks):
        # Project to hidden size
        self.input_proj = PlainLayer(input_size, hidden_size)
        
        # Residual blocks
        self.blocks = [ResidualBlock(hidden_size) for _ in range(num_blocks)]
        
        # Project back to output
        self.output_proj = PlainLayer(hidden_size, input_size)
    
    def forward(self, x):
        x = self.input_proj.forward(x)
        for block in self.blocks:
            x = block.forward(x)
        x = self.output_proj.forward(x)
        return x
    
    def backward(self, dout):
        dout = self.output_proj.backward(dout)
        for block in reversed(self.blocks):
            dout = block.backward(dout)
        dout = self.input_proj.backward(dout)
        return dout

# Create networks
input_size = 16
hidden_size = 16
depth = 10

plain_net = PlainNetwork(input_size, hidden_size, depth)
resnet = ResidualNetwork(input_size, hidden_size, depth)

print(f"Created Plain Network with {depth} layers")
print(f"Created ResNet with {depth} residual blocks")

## Demonstrate Gradient Flow

The key advantage: gradients flow more easily through skip connections

In [ ]:
def measure_gradient_flow(network, name):
    """Measure gradient magnitude at different depths"""
    # Random input
    x = np.random.randn(input_size, 1)
    
    # Forward pass
    output = network.forward(x)
    
    # Create gradient signal
    dout = np.ones_like(output)
    
    # Backward pass
    network.backward(dout)
    
    # Collect gradient magnitudes
    grad_norms = []
    
    if isinstance(network, PlainNetwork):
        for layer in network.layers:
            grad_norm = np.linalg.norm(layer.dW)
            grad_norms.append(grad_norm)
    else:  # ResNet
        grad_norms.append(np.linalg.norm(network.input_proj.dW))
        for block in network.blocks:
            grad_norm1 = np.linalg.norm(block.layer1.dW)
            grad_norm2 = np.linalg.norm(block.layer2.dW)
            grad_norms.append(np.mean([grad_norm1, grad_norm2]))
        grad_norms.append(np.linalg.norm(network.output_proj.dW))
    
    return grad_norms

# Measure gradient flow in both networks
plain_grads = measure_gradient_flow(plain_net, "Plain Network")
resnet_grads = measure_gradient_flow(resnet, "ResNet")

# Plot comparison
plt.figure(figsize=(12, 5))
plt.plot(range(len(plain_grads)), plain_grads, 'o-', label='Plain Network', linewidth=2)
plt.plot(range(len(resnet_grads)), resnet_grads, 's-', label='ResNet', linewidth=2)
plt.xlabel('Layer Depth (deeper →)')
plt.ylabel('Gradient Magnitude')
plt.title('Gradient Flow: ResNet vs Plain Network')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')
plt.show()

print(f"\nPlain Network - First layer gradient: {plain_grads[0]:.6f}")
print(f"Plain Network - Last layer gradient: {plain_grads[-1]:.6f}")
print(f"Gradient ratio (first/last): {plain_grads[0]/plain_grads[-1]:.2f}x\n")

print(f"ResNet - First layer gradient: {resnet_grads[0]:.6f}")
print(f"ResNet - Last layer gradient: {resnet_grads[-1]:.6f}")
print(f"Gradient ratio (first/last): {resnet_grads[0]/resnet_grads[-1]:.2f}x")

print(f"\nResNet maintains gradient flow {(plain_grads[0]/plain_grads[-1]) / (resnet_grads[0]/resnet_grads[-1]):.1f}x better!")

## Visualize Learned Representations

In [ ]:
# Generate synthetic image-like data
def generate_patterns(num_samples=100, size=8):
    """Generate simple 2D patterns"""
    X = []
    y = []
    
    for i in range(num_samples):
        pattern = np.zeros((size, size))
        
        if i % 3 == 0:
            # Horizontal lines
            pattern[2:3, :] = 1
            label = 0
        elif i % 3 == 1:
            # Vertical lines
            pattern[:, 3:4] = 1
            label = 1
        else:
            # Diagonal
            np.fill_diagonal(pattern, 1)
            label = 2
        
        # Add noise
        pattern += np.random.randn(size, size) * 0.1
        
        X.append(pattern.flatten())
        y.append(label)
    
    return np.array(X), np.array(y)

X, y = generate_patterns(num_samples=30, size=4)

# Visualize sample patterns
fig, axes = plt.subplots(1, 3, figsize=(12, 4))
for i, ax in enumerate(axes):
    sample = X[i].reshape(4, 4)
    ax.imshow(sample, cmap='gray')
    ax.set_title(f'Pattern Type {y[i]}')
    ax.axis('off')
plt.show()

print(f"Generated {len(X)} pattern samples")

## Identity Mapping: The Core Insight

**Key Insight**: If identity mapping is optimal, residual should learn F(x) = 0, which is easier than learning H(x) = x

In [ ]:
# Demonstrate identity mapping
x = np.random.randn(hidden_size, 1)

# Initialize residual block
block = ResidualBlock(hidden_size)

# If weights are near zero, F(x) ≈ 0
block.layer1.W *= 0.001
block.layer2.W *= 0.001

# Forward pass
output = block.forward(x)

# Check if output ≈ input (identity)
identity_error = np.linalg.norm(output - x)

print("Identity Mapping Demonstration:")
print(f"Input norm: {np.linalg.norm(x):.4f}")
print(f"Output norm: {np.linalg.norm(output):.4f}")
print(f"Identity error ||F(x) + x - x||: {identity_error:.6f}")
print(f"\nWith near-zero weights, residual block ≈ identity function!")

# Visualize
plt.figure(figsize=(10, 4))
plt.subplot(1, 2, 1)
plt.plot(x.flatten(), 'o-', label='Input x', alpha=0.7)
plt.plot(output.flatten(), 's-', label='Output (x + F(x))', alpha=0.7)
plt.xlabel('Dimension')
plt.ylabel('Value')
plt.title('Identity Mapping: Output ≈ Input')
plt.legend()
plt.grid(True, alpha=0.3)

plt.subplot(1, 2, 2)
residual = output - x
plt.bar(range(len(residual)), residual.flatten())
plt.xlabel('Dimension')
plt.ylabel('Residual F(x)')
plt.title('Learned Residual ≈ 0')
plt.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Compare Network Depths

In [ ]:
def test_depth_scaling():
    """Test how gradient flow scales with depth"""
    depths = [5, 10, 20, 30, 40]
    plain_ratios = []
    resnet_ratios = []
    
    for depth in depths:
        # Create networks
        plain = PlainNetwork(input_size, hidden_size, depth)
        res = ResidualNetwork(input_size, hidden_size, depth)
        
        # Measure gradients
        plain_grads = measure_gradient_flow(plain, "Plain")
        res_grads = measure_gradient_flow(res, "ResNet")
        
        # Calculate ratio (first/last layer gradient)
        plain_ratio = plain_grads[0] / (plain_grads[-1] + 1e-10)
        res_ratio = res_grads[0] / (res_grads[-1] + 1e-10)
        
        plain_ratios.append(plain_ratio)
        resnet_ratios.append(res_ratio)
    
    # Plot
    plt.figure(figsize=(10, 6))
    plt.plot(depths, plain_ratios, 'o-', label='Plain Network', linewidth=2, markersize=8)
    plt.plot(depths, resnet_ratios, 's-', label='ResNet', linewidth=2, markersize=8)
    plt.xlabel('Network Depth')
    plt.ylabel('Gradient Ratio (first/last layer)')
    plt.title('Gradient Flow Degradation with Depth')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.yscale('log')
    plt.show()
    
    print("\nGradient Ratio (first/last) - Higher = Worse gradient flow:")
    for i, d in enumerate(depths):
        print(f"Depth {d:2d}: Plain={plain_ratios[i]:8.2f}, ResNet={resnet_ratios[i]:6.2f} "
              f"(ResNet is {plain_ratios[i]/resnet_ratios[i]:.1f}x better)")

test_depth_scaling()

## Key Takeaways

### The Degradation Problem:
- Adding more layers to plain networks hurts performance
- **Not** due to overfitting (training error also increases)
- Due to optimization difficulty: vanishing/exploding gradients

### ResNet Solution: Skip Connections
```
y = F(x, {Wi}) + x
```

**Instead of learning**: H(x) = desired mapping  
**Learn residual**: F(x) = H(x) - x  
**Then**: H(x) = F(x) + x

### Why It Works:
1. **Identity mapping is easier**: If optimal mapping is identity, F(x) = 0 is easier to learn than H(x) = x
2. **Gradient highways**: Skip connections provide direct gradient paths
3. **Additive gradient flow**: Gradients flow through both residual and skip paths
4. **No extra parameters**: Skip connection is parameter-free

### Impact:
- Enabled 152-layer networks (vs 20-layer limit before)
- Won ImageNet 2015 (3.57% top-5 error)
- Became standard architecture pattern
- Inspired variants: DenseNet, ResNeXt, etc.

### Mathematical Insight:
Gradient of loss L w.r.t. earlier layer:
```
∂L/∂x = ∂L/∂y * (∂F/∂x + ∂x/∂x) = ∂L/∂y * (∂F/∂x + I)
```
The `+ I` term ensures gradients always flow!